In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torch.optim import Adam

import os.path as osp
from tqdm import tqdm, tqdm_notebook
from pytorch_transformers import *

from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.preprocessing import MinMaxScaler

import os
from pathlib import Path
import os.path as osp

from utils import normalize_price
from transformers import BertModel

C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\softwares\conda4.8.2-python3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\softwares\conda4.

In [2]:
import os
import gc
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
from contextlib import contextmanager
import time

import numpy as np
import pandas as pd
import scipy as sp
import random

import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn import preprocessing
import category_encoders as ce
from sklearn.metrics import mean_squared_error

import torch
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
np.random.seed(0)

from pytorch_tabnet.tab_model import TabNetRegressor ##Import Tabnet 
from sklearn.model_selection import ParameterGrid
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

import os
from pathlib import Path
import os.path as osp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

In [3]:
import BertCapsule

In [4]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 17px;
</style>

In [5]:
data_path = osp.join(Path(os.getcwd()).parent,'Data')
model_path = osp.join(Path(os.getcwd()).parent,'Models')

In [6]:
ens_test_df = pd.read_csv(data_path + '/ensemble_test.csv')

In [7]:
dates = ens_test_df['Date'].tolist()

In [8]:
ens_test_df.columns

Index(['Unnamed: 0', 'Date', 'News', 'cap', 'stochaistic_k_percent',
       'stochaistic_D', 'Momentum', 'rate_of_change', 'William_R_percent',
       'AD_oscillator', 'Disparity'],
      dtype='object')

In [9]:
state_dict = torch.load(model_path + '/ensemble-bert-capsule.pth')
#ensemble bert capsule hyper parameters
ebc_model = BertCapsule.bertCapsuleModel(input_dim_capsule=768,
                                     num_capsule=10,dim_capsule=16,
                                     routings=5,kernel_size=(9,1),
                                     dropout_p=0.25,T_epsilon = 1e-7,
                                     batch_size=128)
ebc_model.state_dict = state_dict

In [10]:
from pickle import load
ebc_scaler = load(open(model_path + '/ensemble-bert-capsule-scaler.pkl', 'rb'))

In [11]:
all_headlines = ens_test_df['News'].tolist()
norm_price = ebc_scaler.transform(ens_test_df['cap'].to_numpy().reshape(-1,1))

In [12]:
MAX_LEN = 400
batch_size = 4
input_ids=[]

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in all_headlines]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
for i in tqdm_notebook(range(len(tokenized_texts))):
       input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_texts[i]))        
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
#Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:    
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
test_inputs,test_labels = input_ids, norm_price
test_masks, _ = attention_masks, input_ids
test_inputs =  torch.from_numpy(np.array(test_inputs)).long()
test_labels = torch.from_numpy(np.array(test_labels)).float()
test_masks = torch.from_numpy(np.array(test_masks)).float()
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (701 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    ebc_model = ebc_model.to(device) 

In [14]:
predictions = []
test_labels = []
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_dataloader):
        
        batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
        outputs = ebc_model.forward(b_input_ids,b_input_mask)
      # print (outputs)target.
        test_label = b_labels.cpu().data.numpy().tolist()
        test_labels.append(test_label)
        prediction = outputs.cpu().data.numpy().flatten().tolist()   
        predictions.append(prediction)

output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output shape  torch.Size([1, 1])
output sha

In [15]:
preds = ebc_scaler.inverse_transform(predictions)[:, [0]].flatten()
preds = preds.tolist()

In [16]:
tbr_scaler = load(open(model_path + '/ensemble-tabnet-scaler.pkl', 'rb'))
tbnt_regr = torch.load(model_path + '/ensemble-tabnet.pt')

In [17]:
ens_test_df=ens_test_df.drop(['Unnamed: 0','News','Date'],axis=1)
scaled_features = tbr_scaler.transform(ens_test_df.values)

In [18]:
test = pd.DataFrame(scaled_features, index=ens_test_df.index, columns=ens_test_df.columns)

In [19]:
target = 'cap'
test_indices = test.index
for col in test.columns[test.dtypes == 'float64']:
    test.fillna(test.loc[test_indices, col].mean(), inplace=True)

In [20]:
unused_feat = ['Set']
features = [ col for col in test.columns if col not in unused_feat+[target]] 

In [21]:
X_test = test[features].values[test_indices]
y_test = test[target].values[test_indices].reshape(-1, 1)

In [22]:
tb_predictions = tbnt_regr.predict(X_test)
tb_preds = ebc_scaler.inverse_transform(tb_predictions)[:, [0]].flatten()
tb_preds = tb_preds.tolist()

In [24]:
main_df = pd.DataFrame.from_dict({'date':dates, 'cap' : ens_test_df['cap'],'ebc_pred' : preds ,'tb_pred':tb_preds})

In [27]:
main_df['final_pred'] = -0.0323106 * main_df['ebc_pred'] + 0.09788833 * main_df['tb_pred']

In [28]:
main_df= main_df.drop(['ebc_pred','tb_pred',],axis=1)

In [30]:
result_path = osp.join(Path(os.getcwd()).parent,'Results')
main_df.to_csv(result_path + '/final_pred_date.csv')